In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

#Ignore warnings for now 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/export/sphere/ebirah/ko20929/miniconda3/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [2]:
#Ameneded the error that meant there were 88 values for REM , I can write the TS processing 
#Whilst the generation of the new 6 second frequency data is being done

In [3]:
folder = 'new_freq_ts/'
dtypes =  ['Wake', 'N1', 'N2', 'N3','REM']
results_dict = {}

for data_type in dtypes:
    results_dict[data_type] = {}
    results_dict[data_type]['X'] = pd.read_hdf(folder + data_type + 'six_second_freq_df.h5' , key = 'df', mode = 'r')
    results_dict[data_type]['groups'] = pd.read_hdf(folder + data_type + '_groups.h5' , key = 'df', mode = 'r')
    results_dict[data_type]['y'] = pd.read_hdf(folder + data_type + '_y.h5' , key = 'df', mode = 'r')

In [5]:
for data_type in dtypes:
    print(data_type)
    display(results_dict[data_type]['X'].head(2)) 
    display(results_dict[data_type]['groups'])
    display(results_dict[data_type]['y'])

Wake


,Delta_Central,Theta_Central,Alpha_Central,Sigma_Central,Beta_Central,Gamma_Central,TotalAbsPow_Central,Delta_Frontal,Theta_Frontal,Alpha_Frontal,...,Beta_Right Parietal,Gamma_Right Parietal,TotalAbsPow_Right Parietal,Delta_Right Temporal,Theta_Right Temporal,Alpha_Right Temporal,Sigma_Right Temporal,Beta_Right Temporal,Gamma_Right Temporal,TotalAbsPow_Right Temporal
0,0 0.119395 1 0.115076 2 0.095208 3...,0 0.305156 1 0.240114 2 0.469296 3...,0 0.274594 1 0.358792 2 0.177561 3...,0 0.126387 1 0.111546 2 0.095823 3...,0 0.138452 1 0.143712 2 0.146880 3...,0 0.036015 1 0.030761 2 0.015232 3...,0 49.601829 1 46.612778 2 68.77758...,0 0.078060 1 0.081185 2 0.051734 3...,0 0.283393 1 0.227771 2 0.525861 3...,0 0.293643 1 0.499218 2 0.228874 3...,...,0 0.085936 1 0.097931 2 0.062913 3...,0 0.038117 1 0.043186 2 0.026132 3...,0 75.495552 1 69.002663 2 111.40...,0 0.247600 1 0.190804 2 0.163329 3...,0 0.294811 1 0.448443 2 0.541493 3...,0 0.196601 1 0.183503 2 0.127083 3...,0 0.085892 1 0.056702 2 0.063474 3...,0 0.125665 1 0.080969 2 0.084591 3...,0 0.049432 1 0.039578 2 0.020029 3...,0 57.018125 1 66.747889 2 73.49080...
1,0 0.131017 1 0.239976 2 0.202804 3...,0 0.680105 1 0.519647 2 0.567746 3...,0 0.077806 1 0.093529 2 0.107529 3...,0 0.062556 1 0.063860 2 0.040426 3...,0 0.042362 1 0.073395 2 0.072985 3...,0 0.006153 1 0.009593 2 0.008509 3...,0 43.340905 1 30.903315 2 35.21...,0 0.153110 1 0.206867 2 0.123816 3...,0 0.698610 1 0.540452 2 0.654276 3...,0 0.071543 1 0.084946 2 0.085598 3...,...,0 0.016855 1 0.016113 2 0.028263 3...,0 0.010930 1 0.008341 2 0.013118 3...,0 123.625548 1 140.562024 2 100.99...,0 0.191322 1 0.193031 2 0.213058 3...,0 0.499015 1 0.539117 2 0.533211 3...,0 0.123098 1 0.126665 2 0.090197 3...,0 0.060364 1 0.033554 2 0.061977 3...,0 0.084603 1 0.079962 2 0.067318 3...,0 0.041598 1 0.027671 2 0.034239 3...,0 38.695497 1 53.889329 2 46.04841...


0     1101
1     1102
2     1103
3     1104
4     1105
5     1106
6     1108
7     1109
8     1110
9     1111
10    1112
11    1113
12    1114
13    1115
14    1116
15    1117
16    1118
17    1119
18    2201
19    2202
20    2203
21    2204
22    2204
23    2205
24    2206
25    2207
26    2208
27    2209
28    2210
29    2211
30    2211
31    2212
32    2213
33    2214
34    2215
35    2216
36    4401
37    4401
38    4402
39    4404
40    4404
41    4405
42    4406
43    4407
44    4408
45    4409
46    4410
47    4411
48    3301
49    3302
50    3303
51    3304
52    3304
53    3305
54    3306
55    3306
dtype: object

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    3
19    3
20    3
21    3
22    3
23    3
24    3
25    3
26    3
27    3
28    3
29    3
30    3
31    3
32    3
33    3
34    3
35    3
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    1
47    1
48    2
49    2
50    2
51    2
52    2
53    2
54    2
55    2
dtype: int64

N1


...

0    2
dtype: int64

REM


""


Series([], dtype: float64)

Series([], dtype: int64)